<a href="https://colab.research.google.com/github/ryann-arruda/deep_learning_algorithms/blob/main/perceptron_step_activation_function_and_minibatch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [166]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection  import train_test_split
import pandas as pd
import torch
from torch.func import vmap
from tqdm import tqdm

## **Loading Dataset**

In [167]:
data = load_breast_cancer()

In [168]:
x = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.DataFrame(data.target, columns=['Target'])

## **Separating the Dataset**

In [169]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=27)

## **Converting Data to Tensors**

In [170]:
x_train = torch.tensor(x_train.values, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.float32)

In [171]:
x_test = torch.tensor(x_test.values, dtype=torch.float32)
y_test = torch.tensor(y_test.values, dtype=torch.float32)

## **Perceptron**

In [231]:
class Perceptron():
  def __init__(self, num_features, lr=1.0):
    self.num_featues = num_features
    self.weights = torch.zeros(num_features)
    self.bias = torch.tensor(0.)
    self.lr = torch.tensor(lr)

  def forward(self, x):
    z = self.weights.matmul(x.T) + self.bias

    return torch.where(z >= 0, 1., 0.)

  def fit(self, x, y, epochs, batch_size=1):
    for epoch in tqdm(range(epochs)):
      for chunk_examples, chunk_labels in zip(x.unfold(0, batch_size, batch_size), y.unfold(0, batch_size, batch_size)):
        chunk_examples = chunk_examples.permute(1,0)
        chunk_labels = chunk_labels.permute(1,0)

        a = self.forward(chunk_examples).view(-1, 1)

        dz = a - chunk_labels

        dw = torch.sum(dz * chunk_examples, dim=0)/self.num_featues
        db = torch.sum(dz)/self.num_featues

        self.weights -= self.lr * dw
        self.bias -= self.lr * db

In [232]:
num_features = len(x_train[0])

In [233]:
model = Perceptron(num_features)

In [234]:
model.fit(x_train, y_train, 3000, batch_size=64)

100%|██████████| 3000/3000 [00:02<00:00, 1229.75it/s]


## **Testing**

In [235]:
def accuracy(model, x_test, y_test):
  accuracy = 0.0

  for x, y in zip(x_test, y_test):
    pred = model.forward(x)

    accuracy += (pred == y).item()

  return accuracy/len(x_test)

In [236]:
accuracy(model, x_test, y_test)

0.9415204678362573